## Paired Switching

The idea behind this strategy is that if the assets are negatively correlated, then a traditional mixed portfolio might lead to a lower return than the return for the individual assets.

We periodically compute the Co-relation Coefficients' Matrix and choose pair of stocks which have the most negative co-relations and trade in them. The basic idea being that if one of them would fall, then the other would rise and hence, we must switch between them!

We need a re-balancing period ,i.e. The time after which we would be again calculating the co-relations and taking a different pair of stocks : **T**

We would also need to keep switching between these selected stocks in a pair at a certain interval of time : **P**

Next, We need to specify how many such pairs would we be trading in : **M**

The Fraction of the total balance we would be investing on every turn : **R**

The Brokerage Fee as a fraction of the total transaction amount : **F**



In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

T = 60
P = 7
M = 4
R = 0.8
F = 0.0005

### Few Upcoming Functions(GetData and PartitionData) would be similar to the ones you encountered in the Momentum Assignment (Feel free to Copy-Paste your code from there!)

In the file DATA.csv , which we had uploaded in our repository, we have prices of 30 firms enlisted in S & P 500 Index (Apple, IBM, Cisco, Walmart and the like!) from 2nd January 2009 to 17th August 2020.

For our purposes, We'll only be requiring certain columns. On an honest note, Just getting the Columns on Ticker, Date and Adjusted Closing Price would do the job, but if you want, you may take Opening Price as well.

Read up about the [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function from here, and figure out how you'll use it to do the job (You don't need all the columns!) 

In [2]:
def GetData(NameOfFile):
  #START CODE HERE
  file=pd.read_csv(NameOfFile,usecols=["datadate","tic","adjcp"])
  return(file)  # pd.DataFrame Object

To aid Data-Manipulation, it would be beneficial, if we split the DataFrame into many small parts each corresponding to the data corresponding to the 30 Tickers on a particular date. These small parts could then be stored in a list.

We would also be needing to remember which date is at what index, so that we can use that later. 

In [5]:
def PartitionData(data):
    global DateToIndex
    DateToIndex = {}
    k=0
    #DateToIndex[20090102]=[]
    #print(DateToIndex)
    for index,row in data.iterrows():
        date=data.datadate[index]
        
        if date in DateToIndex.keys():
            pass
            #DateToIndex[date].append(index)
        else:
            DateToIndex[date]=k
            k+=1
            #DateToIndex[date]=[index]
    partitionedlist=[]
    for i in DateToIndex.keys():
        df1 = data[data['datadate'] == i]
        partitionedlist.append(df1)
    
    return(partitionedlist,DateToIndex)
    
    


Now, We need a function which takes as input a pair of stocks (as indices in range 0-29), Today's date,  alongwith the Partitioned DataFrames which will tell us which of the two stocks has a better outlook for the future, and we'll simply move all our Money from one of them to the other. 

This will be done once in every **P** trading sessions.

You could choose between the two stocks in a number of ways, the simplest of which can be what you did in the last assignment, the Momentum Indicator (i.e. Take the difference between the prices of the stocks in a certain number of days and do your deed! Remember this number of days **N** must be <= **T**)

You may also use other indicators like [RSI](https://www.investopedia.com/terms/r/rsi.asp), [Stochs](https://www.investopedia.com/terms/s/stochasticoscillator.asp), [MACD](https://www.investopedia.com/terms/m/macd.asp) or anything else! (If you decide to use MACD or RSI, you could either implement other functions to calculate them or simply change your ParitionData() function to get the MACD and RSI values as they are already stored there for you!)

You need not worry about getting the nearest dates (or dates at all) in this case, because all you care about is co-relations in long term and growth in short-term so it doesn't matter if you're ahead or behind a few days!

'today' would be an index(integer), i.e. an index on the PartitionedDataFrames list denoting where we are at currently!

Also, as you may have figured it out yourself, while DataFrames are great for Data Handling and small statistical calculations, They aren't so for big calculations as the Numpy Library has both a richer variety of functions for such manipulations and is also more efficient!


In [7]:
def Switch(firstStock, SecondStock, today ,PartitionedDataFrames):
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    N=50
    dateformatted=[]
    for i in DateToIndex.keys():
        i=str(i)
        a = datetime.date(int(i[0:4]),int(i[4:6]),int(i[6:]))
        dateformatted.append(a)
    NdaysAgo = datetime.date(int(today[0:4]),int(today[4:6]),int(today[6:])) + datetime.timedelta(days = -N)
    near=min(dateformatted, key=lambda x: abs(x - NdaysAgo))
    nearint = int(near.strftime('%Y%m%d'))
    todayint= int(today)
    #START CODE HERE!
    now=np.array(PartitionedDataFrameList[DateToIndex[todayint]]["adjcp"])
    ago=np.array(PartitionedDataFrameList[DateToIndex[nearint]]["adjcp"])
    momentum = now-ago
    
    #mean
    tot=np.zeros(30)
    values=[]
    for i in range(DateToIndex[nearint],DateToIndex[todayint]+1):
        arr=np.array(PartitionedDataFrameList[i]["adjcp"])
        values.append(arr)
        
        tot=tot+arr
    mean=tot/(DateToIndex[todayint]+1-DateToIndex[nearint])
    final=momentum/mean
    
    if final[firstStock]>final[SecondStock]:
        return(firstStock)
    else:
        return(SecondStock)
    
    '''today=str(today)
    NdaysAgo = datetime.date(int(today[0:4]),int(today[4:6]),int(today[6:])) + datetime.timedelta(days = -N)
    ndays = int(NdaysAgo.strftime('%Y%m%d'))
    
    now=np.array(PartitionedDataFrameList[DateToIndex[int(today)]]["adjcp"])
    ago=np.array(PartitionedDataFrameList[DateToIndex[ndays]]["adjcp"])
    momentum = now-ago
    
    tot=np.zeros(30)
    values=[]
    for i in range(DateToIndex[ndays],DateToIndex[int(today)]+1):
        arr=np.array(PartitionedDataFrameList[i]["adjcp"])
        values.append(arr)
        
        tot=tot+arr
    mean=tot/(DateToIndex[todayint]+1-DateToIndex[nearint])
    final=momentum/mean'''
    
    


Now, We need something to simulate our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp). In order to do that we need a class, which has certain  basic features and functionalities.

(Again some of the functions here, would be same as the previous assignment, so feel free to Copy-Paste)

Features : 


1.   Your Initial Balance
2.   Your Current Balance (Money which isn't invested!)
3.   A list(/any other container) storing the number of stocks of each ticker currently in possession. (Numpy Array prefered)
4.   Most recent prices of all the stocks (As a Numpy array)
5.   A list of **M** or less(as it might be that less than **M** stocks are negatively co-related) Tuples of indices (i,j) where i and j are the indices (from 0-29) of stocks which are negatively co-related.

Functionalities: 



1.   Calculating current Net Worth (Balance+Total Evaluation of all Stocks owned!) 
2.   Buying a Particular Stock (Keep the Transaction fee in mind!)
3.   Selling a particular Stock whole (Keep the Transaction Fee in mind!)
4.   Change Pairs (**T** period intervals) (Takes PartitionedDataFrame, DateT and today's date as input)
5.   Rebalance Portfolio (**P** period intervals) (Takes O)
6.   Function to change the value of most recent prices stored (Takes Numpy array as input)


You may want to know about Numpy's Corrcoef ([np.corrcoef](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html)) function!


In [ ]:
class PortFolio:
    def __init__(self, inibal, currbal, currshares, recprices,pairs):
        self.inibal = inibal
        self.currbal = currbal
        self.currshares = currshares
        self.recprices = recprices
        self.pairs=pairs

    def SellStock(self,index):
        self.currbal = self.currbal + (self.recprices[index] * self.currshares[index]) *(1-F)
        self.currshares[index] = 0

    def BuyStock(self,index, number):
        self.currbal = self.currbal - (self.recprices[index] * number) * (1+F)
        self.currshares[index] = self.currshares[index] + number

    def CalculateNetWorth(self):
        return (self.currbal + np.sum(self.currshares * self.recprices))

    def ChangePricesTo(self,newPriceVector):
        self.recprices = newPriceVector

    def ChangePairs(self,today):
        for tup in self.pairs:
            a = switch(tup[0],tup[1],today,PartitionData(GetData("DATA.csv"))[0])
            if a==tup[0]:
                mone=currshares[tup[1]]*recprices[tup[1]]
                self.SellStock(tup[1])
                self.BuyStock(tup[0],mone/recprices[tup[0]])
            else:
                mone=currshares[tup[0]]*recprices[tup[0]]
                self.SellStock(tup[0])
                self.BuyStock(tup[1],mone/recprices[tup[1]])
        
        '''for i in range(0,30):
            if currshares[i]!=0 :
                for j in range(0,30):
                    if i==j:
                        continue
                    else:
                        if currshares[j]!=0:
                            a=switch(i,j,today,PartitionData(GetData("DATA.csv"))[0])
                            if a==i:
                                mone=currshares[j]*recprices[j]
                                self.SellStock(j)
                                self.BuyStock(i,mone/recprices[i])
                            else:
                                mone=currshares[i]*recprices[i]
                                self.SellStock(i)
                                self.BuyStock(j,mone/recprices[j])'''
                    
                                
            # Calls the Switch function for all the pairs of stocks owned


    def RebalancePortfolio(self):
        self.pairs=[]
        fin=[]
        for j in range(0,30):
            fin.append([])
            for i in PartitionData(GetData("DATA.csv"))[0]:
                fin[j].append(i.iloc[[j]["adjcp"]])
                
        a=np.corrcoeff(fin)
        b=a<-0.7
        for i in range(0,30):
            for j in range(0,30):
                if b[i][j]==1:
                    self.pairs.append((i,j))
        
                
        
            #fin.append(i["adjcp"])
        
            
        
    # Calculates the Co-relation Matrix and changes the list of pairs, which you'll be using to trade
    # You'll have to take care of cases, where a single stock would be involved in two or more of the most negatively co-related Stocks
    # It would be a good idea, to call BuyStock and SellStock from here itself, instead of calling ChangePairs later as it will cause havoc that way
    





With that the difficult part is over!

Now, all you need to work on is a main loop, which calls all these functions

In [ ]:
Data = GetData("Data.csv")
PartitionedData, DateToIndex= PartitionData(Data)


myPortfolio = PortFolio(________)
NetWorthAfterEachTrade = [___]


#


#Start processing from the (T+1)th Day(among the ones recorded in the Data)
for i in range(__________,_________):
  # Change the Prices to the ith Term
  # Get NetWorth and store in list
  # Check if you need to rebalance Portfolio's Today
  # Check if you need to switch stocks today



You may need to declare the PartitionedData list global, depending on your implementation of the Above snippet as well as the various functions in the Portfolio class.

##Moment of Truth

Time to check, if your Program actually works!

Plot the data you collected in various ways (Profits, PercentageProfits etc.) and see if what you did worked!

Feel free to use whichever one of Matplotlib or Seaborn you want to.

Feel free to copy-paste your code from the previous assignment!

In [ ]:
def VizualizeData(________________):








  

You may use this cell to write about your results/performance of the model!